In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "4호모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'4호모델_Q_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
divide_together = tools.dict_data_divide(concat_together7.copy(), divide_what=['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액',
       'BW금액+CB금액+유상증자금액','당기순이익','매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'],
       divide_into="총자본", return_origianl=True)
divide_together["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액/총자본',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액/총자본', 'BW금액+CB금액+유상증자금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
divide_together1 = tools.dict_data_drop(divide_together.copy(),drop_col=['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액',
       'BW금액+CB금액+유상증자금액', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '판매 및 관리비'], dropna=False)
divide_together1["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액/총자본',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액/총자본', 'BW금액+CB금액+유상증자금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본', '최대주주비율',
       '최대주주비율_변동성'],
      dtype='object')

In [18]:
drop_togehter = tools.dict_data_drop(divide_together1.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액/총자본',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액/총자본', 'BW금액+CB금액+유상증자금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본', '최대주주비율',
       '최대주주비율_변동성'],
      dtype='object')

In [19]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 14.07it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 1,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=1)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 27,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=27)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 52,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=52)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 5,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=5)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 89,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=89)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 93,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=93)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 41,
  'model': KMeans(max_iter=500, n

## 모델기록


In [20]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

893.0
414.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,419.0,451.0,425.0,431.0,483.0,538.0,539.0,477.0,555.0,571.0,...,673.0,747.0,888.0,588.0,795.0,680.0,880.0,651.0,648.0,546.0
1,619.0,582.0,630.0,549.0,596.0,634.0,646.0,541.0,649.0,472.0,...,743.0,662.0,623.0,643.0,580.0,601.0,518.0,512.0,828.0,678.0
2,556.0,571.0,552.0,628.0,526.0,433.0,415.0,598.0,414.0,575.0,...,560.0,590.0,503.0,794.0,668.0,775.0,657.0,893.0,571.0,815.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,...,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [21]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [22]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [23]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [24]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8346293935541089
1.0


0.10508934294399155

In [25]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

당기순이익/총자본                                     0.185872
영업이익/총자본                                      0.184095
총자본                                           0.100969
매출총이익/총자본                                     0.096739
판매 및 관리비/총자본                                  0.094428
영업현금흐름/총자본                                    0.091248
최대주주비율                                        0.075834
차입금의존도                                        0.049255
BW금액+CB금액+유상증자금액/총자본                          0.035341
매출채권/총자본                                      0.034729
결산기간_주가변동률                                    0.015169
경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트                0.012922
최대주주비율_변동성                                    0.009674
최대주주변경카운트                                     0.004891
결산기간_거래량증가                                    0.003080
최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액/총자본    0.002406
타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액/총자본           0.002293
상호변경카운트+업종변경카운트                               0.001054
dtype: flo

In [26]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.714286,0.25,0.805556,0.885714,0.909091,0.026316,0.023810,0.930233,0.723404,0.069767,...,0.070707,0.811765,0.080460,0.082353,0.115385,0.084507,0.092308,0.717949,0.133333,0.0
1,0.125000,0.75,0.083333,0.057143,0.090909,0.000000,0.023810,0.046512,0.021277,0.837209,...,0.111111,0.117647,0.091954,0.905882,0.038462,0.126761,0.692308,0.128205,0.800000,0.5
2,0.160714,0.00,0.111111,0.057143,0.000000,0.973684,0.952381,0.023256,0.255319,0.093023,...,0.818182,0.070588,0.827586,0.011765,0.846154,0.788732,0.215385,0.153846,0.066667,0.5


In [27]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.714286,0.25,0.805556,0.885714,0.909091,0.026316,0.023810,0.930233,0.723404,0.069767,...,0.070707,0.811765,0.080460,0.082353,0.115385,0.084507,0.092308,0.717949,0.133333,0.0
1,0.125000,0.75,0.083333,0.057143,0.090909,0.000000,0.023810,0.046512,0.021277,0.837209,...,0.111111,0.117647,0.091954,0.905882,0.038462,0.126761,0.692308,0.128205,0.800000,0.5
2,0.160714,0.00,0.111111,0.057143,0.000000,0.973684,0.952381,0.023256,0.255319,0.093023,...,0.818182,0.070588,0.827586,0.011765,0.846154,0.788732,0.215385,0.153846,0.066667,0.5


In [28]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [29]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,-0.348330,0.014240,0.097653,2,2.5
A000040,0.229130,0.464586,0.426906,0,2.5
A000050,-0.084490,-0.559170,0.351350,1,2.5
A000070,-0.615122,-0.129701,0.341914,2,2.5
A000080,-0.701181,0.491332,0.309099,2,2.5
...,...,...,...,...,...
A353810,-0.708532,0.746095,-0.293623,2,2.5
A354200,0.861246,-0.248269,-0.189247,0,2.5
A357780,-0.873979,-0.232213,-0.293046,2,2.5
A363280,-0.069484,-0.864551,0.112608,1,2.5
